# UDP在特定条件下的生成自由能计算

使用CC方法计算UDP在298K，pH=10，pMg=6的条件下的生成自由能

In [1]:
'''
必要库，在调用本文档的函数之前请确保import这些库
'''
import subprocess
import json
import sys
import os
from typing import Optional, Tuple, Union

import numpy as np
import numpy.typing as npt
from rdkit import Chem

from equilibrator_api import ComponentContribution, Q_

In [2]:
'''
初始化CC类，以便在后续函数中调用
'''
cc = ComponentContribution() # 初始化组分贡献法对象放在全局，避免重复初始化

In [3]:
def get_compound(identifier: str, cc) -> Optional[object]:
    """
    根据标识符获取化合物对象，按优先级尝试多种策略

    注意：调用本函数必须全局初始化cc = ComponentContribution()
    
    Args:
        identifier: 化合物标识符（支持InChI、KEGG、BIGG、Metacyc、SMILES等格式）
        cc: ChemicalCompound 或类似的化合物查询对象
    
    Returns:
        成功时返回化合物对象，失败时返回 None
    """
    def try_get_compound(query: str) -> Optional[object]:
        """尝试获取化合物，失败或返回None时返回None"""
        try:
            result = cc.get_compound(query)
            return result if result is not None else None
        except Exception:
            return None
    
    def is_smiles(s: str) -> bool:
        """判断字符串是否为有效的SMILES格式"""
        try:
            mol = Chem.MolFromSmiles(s)
            return mol is not None
        except Exception:
            return False
    
    def smiles_to_inchi(smiles: str) -> Optional[str]:
        """将SMILES转换为InChI"""
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                inchi = Chem.MolToInchi(mol)
                return inchi
            return None
        except Exception:
            return None
    
    compound = None
    
    # 策略1: InChI
    if identifier.startswith("InChI="):
        try:
            compound = cc.get_compound_by_inchi(identifier)
        except Exception:
            return None
    
    # 策略2: KEGG
    elif identifier.startswith("C") and len(identifier) == 6 and identifier[1:].isdigit():
        compound = try_get_compound(f"kegg:{identifier}")
    
    # 策略3: SMILES
    elif is_smiles(identifier):
        try:
            inchi = smiles_to_inchi(identifier)
            if inchi:
                compound = cc.get_compound_by_inchi(inchi)
            else:
                return None
        except Exception:
            return None
    
    # 策略4 & 5: BIGG 和 Metacyc
    if compound is None:
        # 尝试 BIGG
        compound = try_get_compound(f"bigg.metabolite:{identifier}")
        
        # BIGG 失败，尝试 Metacyc
        if compound is None:
            compound = try_get_compound(f"metacyc.compound:{identifier}")

            # Metacyc 失败，尝试搜索
            if compound is None:
                try:
                    compound = cc.search_compound(identifier)
                except Exception:
                    return None
    
    return compound

In [4]:
def standard_dgf_prime_CC(
    input: str, 
    cc,
    p_h: float = 7.0, 
    p_mg: float = 3.0, 
    I: float = 0.25, 
    T: float = 298.15
) -> Tuple[np.floating, np.floating]:
    '''
    使用组分贡献法(Component Contribution)计算化合物的标准生成自由能

    注意：调用本函数必须先全局初始化cc = ComponentContribution()，同时调用 get_compound() 函数

    参数:
    input: 化合物的InChI字符串或其他Equilibrator API支持的格式
    p_h: 溶液的pH值 (默认值: 7.0)
    p_mg: 溶液的pMg值 (默认值: 3.0)
    I: 离子强度,单位为M (默认值: 0.25M)
    T: 温度,单位为K (默认值: 298.15K)
    
    返回:
    Tuple[np.floating, np.floating]: 
        - standard_dgf_prime_CC: 物质在指定条件下的生成自由能 (Δ_fG'°, kJ/mol)
        - std_CC: 生成自由能误差 (kJ/mol)
    '''
    cc.p_h = Q_(p_h)
    cc.p_mg = Q_(p_mg)
    cc.ionic_strength = Q_(f'{I}M')
    cc.temperature = Q_(f'{T}K')

    # 获取化合物
    cpd = get_compound(input, cc)
    if cpd is None:
        raise ValueError(f"无法找到化合物: {input}")

    # 获取用户指定条件下的生化标准形成自由能 (Δ_fG'°)
    standard_dgf_prime_CC, sigma_fin, sigma_inf = cc.standard_dg_formation(cpd)
    
    # 使用 sigma_fin 作为有限不确定性
    std_CC = np.linalg.norm(sigma_fin) if sigma_fin is not None else np.float64(0.0)
    
    return np.float64(standard_dgf_prime_CC), np.float64(std_CC)

In [5]:
# 计算UDP在298K，pH=10，pMg=6的条件下的生成自由能
p_h = 10
p_mg = 6
T = 298  # K
I = 0.25  # M

# 设置条件
cc.p_h = Q_(p_h)
cc.p_mg = Q_(p_mg)
cc.ionic_strength = Q_(f'{I}M')
cc.temperature = Q_(f'{T}K')

# 使用函数计算UDP的生成自由能
try:
    udp_dgf, udp_std = standard_dgf_prime_CC('UDP', cc, p_h=p_h, p_mg=p_mg, I=I, T=T)
    print(f'UDP在298K，pH=10，pMg=6条件下的生成自由能为: {udp_dgf:.2f} kJ/mol ± {udp_std:.2f} kJ/mol')
except Exception as e:
    print(f"计算UDP生成自由能时出现错误: {e}")
    # 如果上面的方式失败，尝试别的标识符
    try:
        udp_dgf, udp_std = standard_dgf_prime_CC('C00015', cc, p_h=p_h, p_mg=p_mg, I=I, T=T)
        print(f'UDP在298K，pH=10，pMg=6条件下的生成自由能为: {udp_dgf:.2f} kJ/mol ± {udp_std:.2f} kJ/mol')
    except Exception as e2:
        print(f"使用KEGG ID计算也失败: {e2}")
        print("可能需要使用其他形式的UDP标识符或检查化合物是否存在")

UDP在298K，pH=10，pMg=6条件下的生成自由能为: -2096.12 kJ/mol ± 9.03 kJ/mol
